## Swedcham Debit Note Automation

1. Read American Credit Card Invoice
2. Create a new Word File with the current Debit Note
3. Generate a PDF from the Word File
4. Send a mail to Swedcham

#### 1. Read American Credit Card Invoice

In [1]:
# Import Libraries
import PyPDF2
import datetime
import os

In [2]:
# Get American Credit Card Invoice folder
cc_folder = 'D:\\Primeway\\Users Pw\\Oscar.Carvalho\\Contas\\Cartões de Crédito\\American\\'

In [33]:
# Get last invoice file name

# Get current year
today = datetime.date.today()
year = str(today.year)
# print(year)

path = cc_folder + year + '\\'

file_list = [entry.name for entry in os.scandir(path) if entry.name[0:4] == year]
filename = file_list[len(file_list)-1]

print(path)
print(filename)

D:\Primeway\Users Pw\Oscar.Carvalho\Contas\Cartões de Crédito\American\2023\
2023-07-Extrato-American-Bradesco.pdf


In [10]:
# Open PDF file
filename = '2023-07-Extrato-American-Bradesco.pdf'

pdfFileObj = open(filename, 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
print(pdfReader.is_encrypted)

   # print(len(pdfReader.pages))
   # pageObj = pdfReader.pages[0]
   # print(pageObj.extract_text())


True


In [4]:
pdfReader.decrypt('LaylaLaylaLayla')

<PasswordType.OWNER_PASSWORD: 2>

In [5]:
pageObj = pdfReader.pages[0]

In [6]:
print(pageObj.extract_text())

DEMONSTRATIVO MENSAL
AMERICAN EXPRESS GOLD CREDIT
 
Olá, olcscarvalho
12 de Julho de 2023 - 06:00
Extrato de Cartão de Crédito
AGÊNCIA CONTA DATA DE VENCIMENTO
00000 0000000 21/07/2023
Histórico de Lançamentos
OSCAR LUCIANO COSTA S CARVALHO -
AMERICAN EXPRESS GOLD CREDITXXXX.XXXXXX.X9312
DATA HISTÓRICO MOEDA US$ COTAÇÃO
US$R$
21/06/2023 PAG BOLETO BANCARIO -3.674,78
11/07/2023 CUSTO TRANS. EXTERIOR-IOF 14,88
24/04/2023 Drogaria_SP DROGARIA03/03 0,00 0,00 0,0000 56,49
23/05/2023 DROGARIA ONLINE     02/03 0,00 0,00 0,0000 64,23
02/06/2023 FROGPAY*FABIANA PERE02/02 0,00 0,00 0,0000 1.800,00
09/06/2023 DIA BRASIL LJ 1005 0,00 0,00 0,0000 53,32
12/06/2023 DIA BRASIL LJ 1005 0,00 0,00 0,0000 70,97
13/06/2023 MINUTO PA 5864 AV BARA 0,00 0,00 0,0000 68,40
15/06/2023 MERCADOLIVRE*JUBASTOR 0,00 0,00 0,0000 19,89
15/06/2023 DIA BRASIL LJ 1005 0,00 0,00 0,0000 77,51
16/06/2023 MINUTO PA 5864 AV BARA 0,00 0,00 0,0000 34,38
17/06/2023 DIA BRASIL LJ 1005 0,00 0,00 0,0000 34,02
19/06/2023 DIA BRASIL L

In [21]:
import pikepdf 
pdf = pikepdf.open('2023-07-Extrato-American-Bradesco.pdf', password='LaylaLaylaLayla')
len(pdf.pages)

2

In [22]:
page = pdf.pages[0]

In [23]:
instructions = pikepdf.parse_content_stream(page)
data = pikepdf.unparse_content_stream(instructions)
print(data.decode('ascii'))

q
BT
36 806 Td
0 -176 Td
0 -37 Td
0 -16 Td
/F4 10 Tf
<44454d4f4e535452415449564f204d454e53414c> Tj
0 0 Td
0 -16 Td
<414d45524943414e204558505245535320474f4c4420435245444954> Tj
0 0 Td
0 -10 Td
<20> Tj
0 0 Td
0 -23 Td
0 -11.5 Td
0 -19 Td
0 -272 Td
0 -20 Td
0 -65.5 Td
0 -84.53 Td
ET
Q
q
Q
q
2 J
0 G
Q
q
519 0 0 104.3 38 699.7 cm
/img0 Do
Q
BT
1 0 0 1 38 647 Tm
/F1 10 Tf
<4f6ce12c20> Tj
/F2 10 Tf
0.85882 0.17255 0.19216 rg
<6f6c637363617276616c686f> Tj
0 g
ET
BT
1 0 0 1 438.05 636 Tm
/F3 9 Tf
1 0 0 rg
<3132206465204a756c686f2064652032303233202d2030363a3030> Tj
0 g
ET
q
Q
q
2 J
0 G
q
0 J
0 j
1 w
0.87451 0.87451 0.87451 RG
36 629.5 m
559 629.5 l
S
Q
Q
BT
1 0 0 1 38 604 Tm
/F3 12 Tf
<4578747261746f2064652043617274e36f206465204372e96469746f> Tj
ET
q
Q
q
2 J
0 G
Q
BT
1 0 0 1 90.3 541.5 Tm
/F5 7.5 Tf
<4147ca4e434941> Tj
ET
BT
1 0 0 1 156.05 541.5 Tm
/F5 7.5 Tf
<434f4e5441> Tj
ET
BT
1 0 0 1 293.52 541.5 Tm
/F5 7.5 Tf
<444154412044452056454e43494d454e544f> Tj
ET
BT
1 0 0 1 90.3 530 Tm
/F5 7.5 Tf
<